In [0]:
import pandas as pd
from pandas import Series
import numpy
from nltk.tokenize import RegexpTokenizer
import re
import nltk
from nltk import WordPunctTokenizer
from collections import OrderedDict
from fuzzywuzzy import process 
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
import csv
import xlrd
import openpyxl 
from openpyxl import Workbook

#### Parse FDIC text file into list of party names and matter references

In [0]:
print('test')

In [0]:
with open('HKT.txt', 'r') as myfile:
    FDIC_raw = myfile.read()
FDIC_list = re.split('Retention', FDIC_raw)

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
stop_words.append('FORECLOSURE')
stop_words.append('JUDICIAL')
stop_words.append('Institution Name')
stop_words.append('FIN')
stop_words.append('Matter Name')
stop_words.append('System')
stop_words.append('Matter Nbr')
stop_words.append('Date Closed')
stop_words.append('Matter Type')
stop_words.append('LAW')
stop_words.append('RETENTION')
stop_words.append('PERIOD')
stop_words.append('LAW')
stop_words.append('Name')
stop_words.append('HAS')
stop_words.append('ENDED')
stop_words.append('Matter')
stop_words.append('Institution')
stop_words.append('Date')
stop_words.append('Closed')
stop_words.append('LEGH')
stop_words.append('Last')
stop_words.append('Updated')
stop_words.append('Nbr')
stop_words.append('AIC')
stop_words.append('Type')

def normalize_document(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens).lower()
    return doc

cleaned_list = []
for item in FDIC_list:
    clean_item = normalize_document(item)
    cleaned_list.append(clean_item)

#### Write each line of text to a row in csv file

In [0]:
wb = Workbook()
ws = wb.active
column_cell_A = 'A'
ws[column_cell_A+'1'] = 'text'
i = 2
for item in cleaned_list:
    ws[column_cell_A+str(i)] = str(item)
    i = i+1
wb.save("output_FDIC.xlsx")
x =  xlrd.open_workbook('output_FDIC.xlsx')
x1 = x.sheet_by_name('Sheet')
csvfile = open('FD_List.csv', 'w+')
writecsv = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
num = x1.nrows
for rownum in range(x1.nrows): 
    writecsv.writerow(x1.row_values(rownum))
csvfile.close() 

#### Load H&K storage log CSV as DataFrame

In [0]:
df_hk = pd.read_csv('./HK.csv')
df_hk = pd.DataFrame(df_hk).apply(lambda x: x.astype(str).str.lower())
df_hk['Desc'] = df_hk.apply(lambda x :','.join(x.astype(str)),axis=1)
df_hk.head(5000)

#### Load FDIC data into DataFrame

In [0]:
df_FDIC = pd.read_csv("./FD_List.csv")
df_FDIC.head(890)

#### Use FuzzyWuzzy to find best matches for FDIC query terms

In [0]:
choices = df_hk["Desc"]
query = df_FDIC["text"]
wb = Workbook()
ws = wb.active
column_cell_A = 'A'
column_cell_B = 'B'
column_cell_C = 'C'
column_cell_D = 'D'
ws[column_cell_A+'1'] = 'Query'
ws[column_cell_B+'1'] = 'Results'
ws[column_cell_C+'1'] = 'H&K Box Num'
ws[column_cell_D+'1'] = 'Strength of Match'
i = 2
for q in query:
    match = process.extractOne(q,choices=choices,scorer=fuzz.token_sort_ratio, score_cutoff=0)
    if match != None:
        index = match[2]
        box_num = df_hk['HKBoxNum'][index]
        found = match[0]
        strength = match[1]
        print('*******************************')
        print('Looking for:' + '' + q)
        print('*******************************')
        print('Found:' + '' + found)
        print('*******************************')
        print('Box Number' + ' ' + box_num)
        print('*******************************')
        print('Strength' + ' ')
        ws[column_cell_A+str(i)] = q
        ws[column_cell_B+str(i)] = match[0]
        ws[column_cell_C+str(i)] = box_num
        ws[column_cell_D+str(i)] = strength
    i = i+1
wb.save("master.xlsx")
        

*******************************
Looking for:baltimore federal financial baltimore federaldirectors officers rlis c pf director and officer lawsuits
*******************************
Found:baltimore federal financial,vs. cambridge creek dev. corp.,tcf0008586,aw8075
*******************************
Box Number aw8075
*******************************
Looking for:permanent metropolitan fs la attorney rlis c pm malpractice investigationattorney do investigation rlis c po investigationd o
*******************************
Found:north community bank f/k/a metropolitan,confidential investigation,614948414,nan
*******************************
Box Number nan
*******************************
Looking for:permanent altus fsb
*******************************
Found:adler, morris,fdic,nan,nan
*******************************
Box Number nan
*******************************
Looking for:wilson joseph s rlis c lk
*******************************
Found:rtc,wilson concepts - title,9517,9517
*****************************

*******************************
Looking for:american pioneer savings bank magnolia service rlis c nq loan corporation contract cms workoutrestructstlmnt dispute lab
*******************************
Found:watertown savings bank,fdic purchase -first american bank loans,nan,nan
*******************************
Box Number nan
*******************************
Looking for:santos jorge and roxana cms a i real estate not classified for dol
*******************************
Found:marano, thomas,estate planning,active file,nan
*******************************
Box Number nan
*******************************
Looking for:jorge and roxana rlis c lk santos cms hennessey john rlis c la bankruptcy chap rbkrptcypresamda cms adversary
*******************************
Found:rtc industries, inc.,discharge arbitration (herrera and salto),active file,nan
*******************************
Box Number nan
*******************************
Looking for:rich e gene rlis c lk cms dyer riddle mills rlis c lz litigation other pr

*******************************
Looking for:j f association equities inc dale s jones and stephen p simmons cms i i real estate not classified for dol
*******************************
Found:southern california edison,tm/sm:us edison smartconnect in classes 9 and 38,active file,nan
*******************************
Box Number nan
*******************************
Looking for:dupree g darrell and charles tindell vs american pioneer cms c d contract action
*******************************
Found:rtc-conser american pioneer,vs. david cole - foreclosure,tcf0008893,ay6002
*******************************
Box Number ay6002
*******************************
Looking for:american pioneer savings bank sale of stock american rlis c nh corporationsubsidiary pioneer life cms matter
*******************************
Found:watertown savings bank,fdic purchase -first american bank loans,nan,nan
*******************************
Box Number nan
*******************************
Looking for:kay risa l rlis c lk cms
*****

*******************************
Looking for:american pioneer savings bank carrolwood automotive cms a i mixed inc edward brinson paul collateral not classfor dol brinson
*******************************
Found:rtc-conser american pioneer,vs. carrollwood automotive -,tcf0008570,aw8056
*******************************
Box Number aw8056
*******************************
Looking for:onip ltd robert n johnson cms i i real estate not classified for dol
*******************************
Found:f.d.i.c.-park bank of florida,vs. john robert & ethel,tcf0007280,aq1192
*******************************
Box Number aq1192
*******************************
Looking for:rosenbluth stanley and phyllis cms a d suit against association
*******************************
Found:residential contractors association,general employment advice,459070519,8
*******************************
Box Number 8
*******************************
Looking for:american pioneer savings bank park horizon ltd stephen roche alekander j hannigan cms

*******************************
Looking for:apsb homes incvarious homeowners assoc rlis c nz nonlitigation other
*******************************
Found:rtc-rec commonwealth,international apparel assoc.,,502418819,nan
*******************************
Box Number nan
*******************************
Looking for:generalemployee benefit rlis c nz nonlitigation other plans cms
*******************************
Found:heritage community bank,wextrust litigation matter,608476295,nan
*******************************
Box Number nan
*******************************
Looking for:black robert cms c nu ch no asset unsecured
*******************************
Found:blank rome llp,fslic vs. robert c. jacoby,,753990304,nan
*******************************
Box Number nan
*******************************
Looking for:american pioneer savings bank rivera rlis c lz litigation other salvador cms
*******************************
Found:rtc-conser american pioneer,vs. bruce d. & shelby l. reams,tcf0007576,as4797
*************

...

*******************************
Looking for:braden inc oakland lakworth cms a n sale of assets
*******************************
Found:rtc-conser. broadview,sale-lks of newport,8376,8376
*******************************
Box Number 8376
*******************************
Looking for:professional savings bank lake forest park inc cms a n other ongoing non litigation matter or project
*******************************
Found:helm bank usa,bsa/aml/ofac & other regulatory matters,active file,nan
*******************************
Box Number nan
*******************************
Looking for:mitchell james rlis c ln suit on promissory r cms note
*******************************
Found:rtc vs. james r. mitchell,none,489520026,50146
*******************************
Box Number 50146
*******************************
Looking for:south florida apts participations cms a i real estate not classified for dol
*******************************
Found:rafiki solar partners, llc,assistance with fdic investigation,active file,

*******************************
Looking for:security homestead fed sav assoc vinh to cms i i real loi estate not classified for dol asset conversion inc cms i i real estate asset matter
*******************************
Found:birtcher anderson realty, llc,real estate matters,active file,nan
*******************************
Box Number nan
*******************************
Looking for:diedrich darby l rlis c lk kathryn cms
*******************************
Found:rtc-conser security homestead,darby l. & kathryn diedrich,625587419,23372
*******************************
Box Number 23372
*******************************
Looking for:security s la harder hall properties sale cms a n workoutnegotiationdocum entation
*******************************
Found:rtc-conser security s&l assn,sale of harder hall properties,tcf0008365,av9513
*******************************
Box Number av9513
*******************************
Looking for:florida dept of revenue cms a n researchopinion
*******************************
Fo

*******************************
Looking for:palm beach tax collectorbch rlis c nm tax case cms nonjudreal prop
*******************************
Found:rtc/commonwealth federal,associated properties,8771,8771
*******************************
Box Number 8771
*******************************
Looking for:sunbelt savings poveda rlis c lk antonio cms
*******************************
Found:sunbelt savings, fsb,pine brook lake club, ltd.,tcf0008831,ay2060
*******************************
Box Number ay2060
*******************************
Looking for:clayton rlis c lk charley cms
*******************************
Found:rtc/centrust,lease/payton&rachlin,8000,8000
*******************************
Box Number 8000
*******************************
Looking for:cozumel rlis c nd asset sale real apartments cms property
*******************************
Found:rtc-conser sunbelt fed sav,cozumel apartments,tcf0009637,bf0618
*******************************
Box Number bf0618
*******************************
Looking for:h

*******************************
Looking for:vanzant marcus j cms c i real estate asset matter
*******************************
Found:birtcher anderson realty, llc,real estate matters,active file,nan
*******************************
Box Number nan
*******************************
Looking for:the county bank c r corporation of tampa richard glenn salley john and wilda cms c i real estate asset matter
*******************************
Found:security mortgage corporation,mortgage loan to: william n. dortch & patricia a. foley - 190 beacon st., boston, ma,546291986,738001
*******************************
Box Number 738001
*******************************
Looking for:kasten bettina cms c i collectionunsecured note
*******************************
Found:cigna securities, inc.,curtis w. & dolores jensen,tcf0009102,bc0264
*******************************
Box Number bc0264
*******************************
Looking for:floyd edward cms c i real estate asset matter
*******************************
Found:rose,

*******************************
Looking for:bay professional centerub palm bay brevard cnty rlis e nm nonjudreal prop
*******************************
Found:rtc-conser professional fed.,vs. braden river partners,,tcf0008896,ay6005
*******************************
Box Number ay6005
*******************************
Looking for:colonial professional cenetr uepalm bay brevard rlis e nm nonjudreal prop
*******************************
Found:rtc-conser professional fed.,general corporate advice,skt0013657,253994
*******************************
Box Number 253994
*******************************
Looking for:the trust bank roberto garciaesquerro as cms c i real trustee estate asset matter
*******************************
Found:helm bank usa,bsa/aml/ofac & other regulatory matters,active file,nan
*******************************
Box Number nan
*******************************
Looking for:the trust bank covent michael j md pa cms c i real and estate asset matter catherine
*******************************
